# Empirical Bayes Shrinkage for Categorical Distributions


## Framework Components
1. Aggregate counts to county × landcover × category
2. Estimate landcover-level baseline distributions
3. Apply exposure-aware shrinkage
4. Output stabilized proportions and diagnostics

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [10]:
# Load dataset
DATA_PATH = Path('../../dataset/Capstone2025_nsi_lvl9_with_landcover_and_color.csv.gz')
df = pd.read_csv(DATA_PATH, compression='gzip', low_memory=False)

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")

Dataset shape: (2417766, 8)

Columns: ['h3', 'fips', 'st_damcat', 'bldgtype', 'lc_type', 'loc', 'clr', 'clr_cc']


## Step 1: Aggregate to County × Landcover × Category

Aggregate structure counts to the county × landcover × category level. This is the unit at which we'll apply shrinkage.


- For each combination of county $i$, landcover type $j$, and category $k$, we sum the structure counts
- This produces the observed counts $n_{i,j,k}$ and total exposure $N_{i,j}$ 

In [13]:
def aggregate_counts(df, group_cols, count_col='clr_cc', category_col='clr'):
    """
    Aggregate counts to county × landcover × category level.
    group_cols : list
        Columns to group by (e.g., ['fips', 'lc_type'])
    count_col : str
        Column containing counts (default: 'clr_cc')
    category_col : str
        Column containing categorical values (default: 'clr')
    
    Returns:
    --------
    DataFrame with columns: group_cols + [category_col, 'count', 'exposure']
    """
    # Aggregate counts
    agg_df = df.groupby(group_cols + [category_col])[count_col].sum().reset_index()
    agg_df.rename(columns={count_col: 'count'}, inplace=True)
    # Calculate exposure (total structures per group)
    exposure = agg_df.groupby(group_cols)['count'].sum().reset_index()
    exposure.rename(columns={'count': 'exposure'}, inplace=True)
    # Merge exposure back
    agg_df = agg_df.merge(exposure, on=group_cols)
    
    return agg_df
# Aggregate to county × landcover × color
counts_df = aggregate_counts(df, group_cols=['fips', 'lc_type'], 
                              count_col='clr_cc', category_col='clr')


print(counts_df.head(10))
print(f"\nExposure summary:")
print(counts_df.groupby(['fips', 'lc_type'])['exposure'].first().describe())

   fips lc_type        clr  count  exposure
0  6001  forest  alabaster    891     10026
1  6001  forest      amber    579     10026
2  6001  forest      cocoa   3656     10026
3  6001  forest      green   1843     10026
4  6001  forest       grey      1     10026
5  6001  forest     indigo      1     10026
6  6001  forest       navy   1063     10026
7  6001  forest     orange   1213     10026
8  6001  forest        red    779     10026
9  6001   grass  alabaster      9        34

Exposure summary:
count    4.700000e+02
mean     2.069838e+04
std      1.037769e+05
min      2.000000e+00
25%      1.997500e+02
50%      1.415500e+03
75%      8.628500e+03
max      1.891556e+06
Name: exposure, dtype: float64


### Mathematical Formulation: Aggregation

**Variables:**
- $i$: County index (`fips` column)
- $j$: Landcover type index (`lc_type` column)
- $k$: Category index (`clr` column)

**Observed Counts:**
$$n_{i,j,k} = \sum_{\text{records in } (i,j,k)} \text{count}$$

This corresponds to the **`count`** column in the aggregated dataframe: the number of structures for county $i$, landcover type $j$, and category $k$.

**Total Exposure:**
$$N_{i,j} = \sum_{k=1}^{K} n_{i,j,k}$$

This corresponds to the **`exposure`** column in the aggregated dataframe: the total number of structures for county $i$ and landcover type $j$ (summed across all categories $k$). Low exposure means high variance in proportions, motivating shrinkage.

**Observed Proportion:**
$$\pi_{i,j,k} = \frac{n_{i,j,k}}{N_{i,j}} = \frac{\text{count}}{\text{exposure}}$$

By construction, $\sum_{k=1}^{K} \pi_{i,j,k} = 1$ for each $(i,j)$ group.

**Connection to Next Step:** These aggregated counts $n_{i,j,k}$ (stored as `count`) and exposure $N_{i,j}$ (stored as `exposure`) will be used to compute baseline distributions $\pi_{0,j,k}$ (aggregated across counties) and then apply shrinkage based on exposure levels.

## Step 2: Compute Landcover-Specific Baseline Distributions

The baseline is the overall distribution of categories within each landcover type, aggregated across all counties. This serves as the prior distribution for shrinkage.


- For each landcover type $j$ and category $k$, aggregate counts across all counties $i$ to compute the baseline proportion $\pi_{0,j,k}$
- This baseline represents the "typical" distribution expect to see, and will be used as the prior in the shrinkage formula

### Mathematical Formulation: Baseline Distribution

**Baseline Proportion:**
$$\pi_{0,j,k} = \frac{\sum_{i=1}^{I} n_{i,j,k}}{\sum_{i=1}^{I} N_{i,j}}$$

The baseline $\pi_{0,j,k}$ represents the overall proportion of category $k$ within landcover type $j$, aggregated across all counties. This serves as the **prior distribution** for shrinkage.

By construction, $\sum_{k=1}^{K} \pi_{0,j,k} = 1$ for each landcover type $j$.

The baseline $\pi_{0,j,k}$ will be used as the prior in the shrinkage formula. When a county has low exposure, its observed proportions will be pulled toward these landcover-specific baselines.

In [ ]:
def compute_baseline_distributions(counts_df, group_cols, category_col='clr'):
    """
    Compute baseline (prior) distributions for each landcover type.
    
    Parameters:
    -----------
    counts_df : DataFrame
        Aggregated counts dataframe
    group_cols : list
        Columns that define groups (e.g., ['fips', 'lc_type'])
    category_col : str
        Column containing categorical values
    
    Returns:
    --------
    DataFrame with columns: [landcover_col] + [category_col] + ['baseline_prop']
    """
    # Extract landcover column (assumed to be 'lc_type')
    landcover_col = 'lc_type'
    
    # Aggregate across all counties for each landcover × category
    baseline = counts_df.groupby([landcover_col, category_col])['count'].sum().reset_index()
    
    # Calculate proportions within each landcover type
    landcover_totals = baseline.groupby(landcover_col)['count'].sum().reset_index()
    landcover_totals.rename(columns={'count': 'total'}, inplace=True)
    
    baseline = baseline.merge(landcover_totals, on=landcover_col)
    baseline['baseline_prop'] = baseline['count'] / baseline['total']
    
    # Keep only necessary columns
    baseline = baseline[[landcover_col, category_col, 'baseline_prop']]
    
    return baseline

# Compute baselines
baseline_df = compute_baseline_distributions(counts_df, group_cols=['fips', 'lc_type'])

print(f"Baseline distributions shape: {baseline_df.shape}")
print(f"\nSample baseline distributions:")
print(baseline_df.head(15))
print(f"\nBaseline proportions sum to 1.0 per landcover:")
print(baseline_df.groupby('lc_type')['baseline_prop'].sum().head(10))

Baseline distributions shape: (419, 3)

Sample baseline distributions:
   lc_type         clr  baseline_prop
0   barren   alabaster       0.102273
1   barren       amber       0.041921
2   barren        aqua       0.001982
3   barren  aquamarine       0.008519
4   barren      auburn       0.000084
5   barren       azure       0.001898
6   barren         bar       0.000084
7   barren       beige       0.009911
8   barren        blue       0.000801
9   barren       brown       0.006495
10  barren       cocoa       0.413690
11  barren      coffee       0.003585
12  barren     emerald       0.000127
13  barren         foo       0.006157
14  barren        gold       0.000127

Baseline proportions sum to 1.0 per landcover:
lc_type
barren          1.0
crop            1.0
forest          1.0
grass           1.0
other           1.0
shrub           1.0
urban           1.0
urban+barren    1.0
urban+crop      1.0
urban+forest    1.0
Name: baseline_prop, dtype: float64


### Mathematical Formulation: Shrinkage Weight and Stabilized Proportion

**Prior Strength Parameter:**
- $\alpha$: Prior strength parameter (default: $\alpha = 10.0$)
- Controls the trade-off between observed data and baseline prior

**Shrinkage Weight:**
$$w_{i,j} = \frac{N_{i,j}}{N_{i,j} + \alpha}$$

- Low exposure ($N_{i,j} \ll \alpha$): $w_{i,j} \approx 0$ → strong shrinkage toward baseline
- High exposure ($N_{i,j} \gg \alpha$): $w_{i,j} \approx 1$ → minimal shrinkage, trust observed data
- When $N_{i,j} = \alpha$: $w_{i,j} = 0.5$ (equal weight)

**Stabilized Proportion:**
$$\tilde{\pi}_{i,j,k} = w_{i,j} \cdot \pi_{i,j,k} + (1 - w_{i,j}) \cdot \pi_{0,j,k}$$

This is a weighted average of observed and baseline proportions. Substituting the shrinkage weight:

$$\tilde{\pi}_{i,j,k} = \frac{N_{i,j}}{N_{i,j} + \alpha} \cdot \pi_{i,j,k} + \frac{\alpha}{N_{i,j} + \alpha} \cdot \pi_{0,j,k}$$

**Movement Metric:**
$$\Delta_{i,j,k} = \tilde{\pi}_{i,j,k} - \pi_{i,j,k}$$

Quantifies how much the proportion changed due to shrinkage. Large absolute movement indicates strong shrinkage (low exposure).

**Effective Sample Size:**
$$N_{\text{eff}} = N_{i,j} + \alpha$$

Represents the combined information from observed data and prior.

**Connection to Next Step:** The stabilized proportions will be validated to ensure they behave as expected (low-exposure groups shrink more, proportions sum correctly, variance is reduced).

## Step 3: Apply Exposure-Aware Shrinkage

**What we're computing:**
- For each county $i$, landcover type $j$, and category $k$, we compute:
  1. **Shrinkage weight** $w_{i,j} = \frac{N_{i,j}}{N_{i,j} + \alpha}$ (exposure-dependent)
  2. **Stabilized proportion** $\tilde{\pi}_{i,j,k} = w_{i,j} \cdot \pi_{i,j,k} + (1 - w_{i,j}) \cdot \pi_{0,j,k}$ (weighted average)
  3. **Movement** $\Delta_{i,j,k} = \tilde{\pi}_{i,j,k} - \pi_{i,j,k}$ (diagnostic metric)

The parameter $\alpha$ (prior strength) controls how much we trust the baseline vs observed data. Higher values mean more shrinkage toward baseline.

### Validation: Proportion Sums

**Observed Proportions:**
$$\sum_{k=1}^{K} \pi_{i,j,k} = \sum_{k=1}^{K} \frac{n_{i,j,k}}{N_{i,j}} = \frac{N_{i,j}}{N_{i,j}} = 1$$

By construction, observed proportions always sum to exactly 1.0.

**Stabilized Proportions:**
$$\sum_{k=1}^{K} \tilde{\pi}_{i,j,k} = w_{i,j} \sum_{k=1}^{K} \pi_{i,j,k} + (1 - w_{i,j}) \sum_{k=1}^{K} \pi_{0,j,k} = 1$$

In theory, stabilized proportions should also sum to 1.0. However, deviations can occur when:
- Not all categories are observed in a group (some $n_{i,j,k} = 0$)
- The baseline includes categories not present in the observed data
- Shrinkage pulls toward baseline categories that weren't observed

Low-exposure groups may show sums < 1.0 because shrinkage pulls toward baseline categories that weren't observed in that specific group. This is acceptable as long as deviations are small (< 0.05) and occur primarily in very low-exposure groups.

In [ ]:
def apply_shrinkage(counts_df, baseline_df, prior_strength=10.0, 
                   group_cols=['fips', 'lc_type'], category_col='clr'):
    """
    Apply empirical Bayes shrinkage to stabilize categorical proportions.
    
    Parameters:
    -----------
    counts_df : DataFrame
        Aggregated counts with columns: group_cols + [category_col, 'count', 'exposure']
    baseline_df : DataFrame
        Baseline distributions with columns: ['lc_type', category_col, 'baseline_prop']
    prior_strength : float
        Strength of prior (baseline). Higher values = more shrinkage toward baseline.
        Default: 10.0 (moderate shrinkage)
    group_cols : list
        Columns that define groups
    category_col : str
        Column containing categorical values
    
    Returns:
    --------
    DataFrame with raw and stabilized proportions, plus diagnostics
    """
    # Extract landcover column
    landcover_col = 'lc_type'
    
    # Calculate observed proportions within each group
    result = counts_df.copy()
    result['observed_prop'] = result['count'] / result['exposure']
    
    # Merge baseline proportions
    result = result.merge(baseline_df, on=[landcover_col, category_col], how='left')
    
    # Fill missing baselines with uniform prior (shouldn't happen, but safety check)
    result['baseline_prop'] = result['baseline_prop'].fillna(1.0 / result[category_col].nunique())
    
    # Calculate shrinkage weight
    # Higher exposure → higher weight → less shrinkage
    result['shrinkage_weight'] = result['exposure'] / (result['exposure'] + prior_strength)
    
    # Apply shrinkage
    result['stabilized_prop'] = (
        result['shrinkage_weight'] * result['observed_prop'] + 
        (1 - result['shrinkage_weight']) * result['baseline_prop']
    )
    

    result['movement'] = result['stabilized_prop'] - result['observed_prop']
    result['abs_movement'] = np.abs(result['movement'])
    

    result['effective_n'] = result['exposure'] + prior_strength
    
    return result

# Apply shrinkage with default prior_strength
prior_strength = 10.0
shrinkage_result = apply_shrinkage(counts_df, baseline_df, prior_strength=prior_strength)

print(f"Shrinkage result shape: {shrinkage_result.shape}")
print(f"\nSample results (showing shrinkage effect):")
sample = shrinkage_result[
    shrinkage_result['exposure'] < 20  
].head(10)
print(sample[['fips', 'lc_type', 'clr', 'exposure', 'observed_prop', 
              'baseline_prop', 'stabilized_prop', 'movement', 'shrinkage_weight']])

Shrinkage result shape: (4493, 12)

Sample results (showing shrinkage effect):
     fips       lc_type     clr  exposure  observed_prop  baseline_prop  \
107  6003  urban+forest    aqua        16       0.062500       0.001207   
108  6003  urban+forest  coffee        16       0.125000       0.044642   
109  6003  urban+forest   ivory        16       0.062500       0.010267   
110  6003  urban+forest   lilac        16       0.062500       0.011020   
111  6003  urban+forest   olive        16       0.062500       0.076139   
112  6003  urban+forest  orange        16       0.062500       0.040741   
113  6003  urban+forest     tan        16       0.562500       0.000749   
114  6003   urban+shrub   ivory         9       0.111111       0.033588   
115  6003   urban+shrub   lilac         9       0.222222       0.041721   
116  6003   urban+shrub  orange         9       0.222222       0.092016   

     stabilized_prop  movement  shrinkage_weight  
107         0.038926 -0.023574          0.61

#### Validation: Exposure-Dependent Shrinkage Behavior

Apply stronger shrinkage to low-exposure groups.

**Expected Pattern:**
- **Low exposure** (< 10 structures): High absolute movement ($|\Delta|$), low shrinkage weight ($w$)
- **Medium exposure** (10-50 structures): Moderate movement, moderate shrinkage weight
- **High exposure** (> 100 structures): Minimal movement, shrinkage weight near 1.0


For a group with exposure $N_{i,j}$ and prior strength $\alpha$:

- **Shrinkage weight:** $w_{i,j} = \frac{N_{i,j}}{N_{i,j} + \alpha}$
  - When $N_{i,j} \ll \alpha$: $w_{i,j} \approx 0$ (strong shrinkage)
  - When $N_{i,j} \gg \alpha$: $w_{i,j} \approx 1$ (minimal shrinkage)

- **Absolute movement:** $|\Delta_{i,j,k}| = |\tilde{\pi}_{i,j,k} - \pi_{i,j,k}|$
  - Larger when $w$ is small (more shrinkage)
  - Smaller when $w$ is large (less shrinkage)

**Interpretation:** The results show that groups with exposure < 5 have mean absolute movement of 0.31 (substantial shrinkage), while groups with exposure > 100 have mean movement of 0.0009 (essentially unchanged). This confirms the method is working as intended.

#### Diagnostics and Validation



1. **Proportion Sums**: Check that $\sum_k \pi_{i,j,k} = 1$ and $\sum_k \tilde{\pi}_{i,j,k} \approx 1$ for each group $(i,j)$
2. **Exposure Dependence**: Verify that low-exposure groups show higher $|\Delta_{i,j,k}|$ (more shrinkage)
3. **Variance Reduction**: Confirm that $\text{Var}(\tilde{\pi}_{i,j,\cdot}) < \text{Var}(\pi_{i,j,\cdot})$ for medium-exposure groups

These diagnostics ensure the shrinkage mechanism is working as intended: stabilizing low-exposure groups while preserving high-exposure groups.

#### Validation: Variance Reduction

One of the primary goals of shrinkage is to reduce variance in proportion estimates, especially for low-exposure groups where observed proportions are highly variable.

**Variance Calculation:**

For each county × landcover group $(i,j)$, we compute the variance across categories:

- **Raw variance:** $\text{Var}(\pi_{i,j,\cdot}) = \frac{1}{K-1} \sum_k (\pi_{i,j,k} - \bar{\pi}_{i,j})^2$
- **Stabilized variance:** $\text{Var}(\tilde{\pi}_{i,j,\cdot}) = \frac{1}{K-1} \sum_k (\tilde{\pi}_{i,j,k} - \bar{\tilde{\pi}}_{i,j})^2$

where $K$ is the number of categories and $\bar{\pi}_{i,j} = \frac{1}{K}\sum_k \pi_{i,j,k}$ is the mean proportion.

**Variance Reduction:**
$$\text{Variance Reduction} = \frac{\text{Var}(\pi_{i,j,\cdot}) - \text{Var}(\tilde{\pi}_{i,j,\cdot})}{\text{Var}(\pi_{i,j,\cdot})}$$

This measures the percentage reduction in variance. Positive values indicate successful variance reduction.

**Expected Pattern:** 
- **Low exposure:** High variance reduction (shrinkage pulls extreme proportions toward baseline)
- **High exposure:** Low variance reduction (observed proportions are already stable)

**Note on Extreme Values:** For very low exposure groups (< 10), variance reduction can show extreme values (including negative infinity) because:
- Observed variance may be near zero (only one or two categories observed)
- Division by near-zero variance causes numerical instability
- This is expected and acceptable for these edge cases

**Interpretation:** The results show substantial variance reduction (24-45%) for medium-exposure groups (10-50 structures), confirming that shrinkage successfully stabilizes estimates without over-shrinking high-exposure groups.

In [ ]:

prop_sums = shrinkage_result.groupby(['fips', 'lc_type']).agg({
    'observed_prop': 'sum',
    'stabilized_prop': 'sum'
}).reset_index()

print("Proportion sums (should be ~1.0):")
print(prop_sums.describe())
print(f"\nGroups where proportions don't sum to 1.0 (±0.01 tolerance):")
print(prop_sums[
    (prop_sums['observed_prop'] < 0.99) | (prop_sums['observed_prop'] > 1.01) |
    (prop_sums['stabilized_prop'] < 0.99) | (prop_sums['stabilized_prop'] > 1.01)
].head(10))

Proportion sums (should be ~1.0):
              fips  observed_prop  stabilized_prop
count   470.000000   4.700000e+02       470.000000
mean   6058.523404   1.000000e+00         0.944411
std      32.938958   2.292656e-17         0.137035
min    6001.000000   1.000000e+00         0.178694
25%    6029.500000   1.000000e+00         0.969180
50%    6059.000000   1.000000e+00         0.996087
75%    6085.000000   1.000000e+00         0.999408
max    6115.000000   1.000000e+00         1.000000

Groups where proportions don't sum to 1.0 (±0.01 tolerance):
    fips       lc_type  observed_prop  stabilized_prop
1   6001         grass            1.0         0.925083
5   6001    urban+crop            1.0         0.943792
8   6001   urban+other            1.0         0.908262
10  6003        forest            1.0         0.984435
11  6003         shrub            1.0         0.694083
12  6003  urban+forest            1.0         0.686448
13  6003   urban+shrub            1.0         0.561952
14  6

In [ ]:
exposure_bins = [0, 5, 10, 20, 50, 100, np.inf]
exposure_labels = ['<5', '5-10', '10-20', '20-50', '50-100', '100+']
shrinkage_result['exposure_bin'] = pd.cut(shrinkage_result['exposure'], 
                                          bins=exposure_bins, labels=exposure_labels)

shrinkage_by_exposure = shrinkage_result.groupby('exposure_bin').agg({
    'abs_movement': 'mean',
    'shrinkage_weight': 'mean',
    'exposure': 'mean',
    'fips': 'nunique'
}).round(4)

print("Shrinkage statistics by exposure level:")
print(shrinkage_by_exposure)


Shrinkage statistics by exposure level:
              abs_movement  shrinkage_weight    exposure  fips
exposure_bin                                                  
<5                  0.3071            0.2392      3.2581    13
5-10                0.1158            0.4404      8.0000     5
10-20               0.0567            0.6083     15.8023    13
20-50               0.0212            0.7735     35.9315    24
50-100              0.0095            0.8852     79.2857    20
100+                0.0009            0.9891  30186.6167    58


In [ ]:
# Compare raw vs stabilized variance
variance_comparison = shrinkage_result.groupby(['fips', 'lc_type']).agg({
    'observed_prop': 'var',
    'stabilized_prop': 'var',
    'exposure': 'first'
}).reset_index()
variance_comparison.columns = ['fips', 'lc_type', 'raw_variance', 'stabilized_variance', 'exposure']

# Calculate variance reduction
variance_comparison['variance_reduction'] = (
    (variance_comparison['raw_variance'] - variance_comparison['stabilized_variance']) / 
    variance_comparison['raw_variance']
).fillna(0)

print("Variance reduction by exposure level:")
variance_comparison['exposure_bin'] = pd.cut(variance_comparison['exposure'], 
                                            bins=exposure_bins, labels=exposure_labels)
print(variance_comparison.groupby('exposure_bin').agg({
    'variance_reduction': 'mean',
    'exposure': 'mean'
}).round(4))

Variance reduction by exposure level:
              variance_reduction    exposure
exposure_bin                                
<5                          -inf      3.0000
5-10                        -inf      8.3333
10-20                     0.4512     15.6000
20-50                     0.2401     35.4000
50-100                    0.1413     78.7391
100+                      0.0185  25525.0315


In [11]:

RESULTS_TABLES_DIR = Path('../../results/tables')
RESULTS_TABLES_DIR.mkdir(parents=True, exist_ok=True)


counts_df.to_csv(RESULTS_TABLES_DIR / 'bayesian_shrinkage_aggregated_counts.csv', index=False)
baseline_df.to_csv(RESULTS_TABLES_DIR / 'bayesian_shrinkage_baseline_distributions.csv', index=False)
shrinkage_result.to_csv(RESULTS_TABLES_DIR / 'bayesian_shrinkage_stabilized_distributions.csv', index=False)


### Next Steps

1. Tune `prior_strength` parameter based on validation
2. Apply stabilized distributions to downstream anomaly detection
3. Compare raw vs stabilized anomaly scores

## Intuitive Explanation: What's Happening Here?

**The Problem:** We have structure color data (`clr` column) across California counties (`fips` column) and landcover types (`lc_type` column). For each county × landcover combination, we want to know the true distribution of colors, but some combinations have very few structures (low exposure), making the observed proportions unreliable.

**The Solution:** Bayesian shrinkage stabilizes these estimates by pulling unreliable proportions toward landcover-specific baselines.

**How It Works:**

1. **Aggregation**: We count structures (`clr_cc`) for each county (`fips`) × landcover (`lc_type`) × color (`clr`) combination. This gives us observed counts $n_{i,j,k}$ and total exposure $N_{i,j}$.

2. **Baseline**: For each landcover type, we compute the overall color distribution across all counties. For example, if barren landcover typically has 41% cocoa-colored structures statewide, that becomes our baseline $\pi_{0,j,k}$ for barren landcover.

3. **Shrinkage**: For each county × landcover group:
   - **High exposure** (many structures): We trust the observed proportions. The stabilized proportion $\tilde{\pi}_{i,j,k}$ stays close to what we observed.
   - **Low exposure** (few structures): We don't trust the observed proportions. The stabilized proportion gets pulled toward the landcover-specific baseline.

**Why This Matters:** Without shrinkage, a county with only 3 structures in a particular landcover type might show 100% of one color just by chance, creating spurious "anomalies." Shrinkage smooths these out by recognizing that counties with the same landcover type should have similar color distributions, especially when we have little data.

**The Result:** We get stabilized color distributions that are more reliable for downstream analysis, especially for rare county × landcover combinations where raw counts would be too noisy.